In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
from PIL import Image
import os
from tqdm import tqdm
import cv2
import time
import torchvision
import torch
import torchvision.transforms as transforms
import sys


import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler
import torch.optim as optim
from tqdm import tqdm_notebook, tnrange

import torch.utils.model_zoo as model_zoo
import torchvision.models as models
from torchvision import datasets
from tqdm.notebook import *
from torchcontrib.optim import SWA

import time
import matplotlib.pyplot as plt


transform = transforms.Compose([
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.43216, 0.394666, 0.37645],
                            std=[0.22803, 0.22145, 0.216989])
])
trans1 = transforms.ToPILImage()

from evaluation import *
from complementary import *
from Myutils import *
from training_engine import *
import optuna
import random

In [3]:
tab = pd.read_csv('/media/aous/Samsung_T5/mtrain.csv')

In [6]:
tab[tab.negative_exam_for_pe == 0]['StudyInstanceUID'].nunique()

2368

In [7]:
tab['StudyInstanceUID'].nunique()

7279

In [2]:
def Resnet50(num_classes):
    net = torchvision.models.resnet50(pretrained=True, progress=True)
    out_size = net.fc.in_features
    net.fc = nn.Linear(out_size, num_classes)
    return(net)
def Resnet152(num_classes):
    net = torchvision.models.resnet152(pretrained=True, progress=True)
    out_size = net.fc.in_features
    net.fc = nn.Linear(out_size, num_classes)
    return(net)

In [3]:
def form_batch(inpt_slices, outpt_size, size, augment):
    inpt_slices.sort_values(['path'], inplace = True)
    transform = transforms.Compose([
                  transforms.ToTensor(),
                  transforms.Normalize(mean=[0.43216, 0.394666, 0.37645],
                                        std=[0.22803, 0.22145, 0.216989])
                ])
    outpt = torch.zeros((3, outpt_size, size[0], size[1]))
    rot = (np.random.random()*2-1)*15
    c = int((np.random.random()*2-1)*size[0]*0.15)
    f = int((np.random.random()*2-1)*size[1]*0.15)
    for i, idx in enumerate(inpt_slices.index):
        im = Image.open(inpt_slices['path'][idx])
        if augment:
            im = im.rotate(rot, translate = (c,f))
        im_vect = transform(im)
        outpt[0][i] = im_vect[0]
        outpt[1][i] = im_vect[1]
        outpt[2][i] = im_vect[2]
    return(outpt)

In [4]:
def reduce_resolution_CT(inpt_exam, outpt_size):
    if len(inpt_exam) < outpt_size:
        return(inpt_exam)
    result = pd.DataFrame(columns = inpt_exam.columns)
    temp = inpt_exam.reset_index().drop(['index'], axis = 1)
    fact = int((len(inpt_exam)*10/outpt_size))
    rm_count = fact - 10
    #remove rm_count element from every fact batch
    step = int(len(inpt_exam)/fact)
    for i in range(step):
        batch_t = temp[temp.index <= (i + 1)*fact]
        batch = batch_t[batch_t.index > i*fact]
        #rm_batch = batch[batch.pe_present_on_image == 0]
        rm_batch = batch
        if len(rm_batch) >= rm_count:
            rm_idx = random.sample(list(rm_batch.index), rm_count)
            batch.drop(rm_idx, axis = 0, inplace = True)
            #print(rm_idx)
        else:
            batch.drop(rm_batch.index, axis = 0, inplace = True)
            rm_idx = random.sample(list(batch.index), rm_count - len(rm_batch))
            batch.drop(rm_idx, axis = 0, inplace = True)
            #print(rm_idx)
        result = pd.concat([result, batch]).reset_index().drop(['index'], axis = 1)
    if len(result) == outpt_size -1:
        result = pd.concat([result, result[len(result)-1:]]).reset_index().drop(['index'], axis = 1)
    return(result)

In [5]:
def distinct_slices(inpt_exam, window_size):
    temp = inpt_exam.copy().reset_index().drop(['index'], axis=1)
    temp['slice'] = inpt_exam['SOPInstanceUID']
    step_num = int(len(inpt_exam)/window_size)
    idx = 0
    for i in range(step_num):
        instance = temp['SOPInstanceUID'][i*window_size]
        for j in range(window_size):
            idx = i*window_size + j
            temp['slice'][idx] = instance
    if idx < len(inpt_exam)-1:
        idx += 1
        instance = temp['SOPInstanceUID'][idx]
        temp['slice'][idx] = instance
        for j in range(len(inpt_exam) - idx):
            idx += 1
            temp['slice'][idx] = instance
    return(temp)

In [6]:
def table_3D(table, max_resol, window_size):
    Exams = list(table['StudyInstanceUID'].unique())
    #reduce resolution
    l_res = pd.DataFrame(columns = table.columns)
    l_res['slice'] = l_res['SOPInstanceUID']
    for i, Exam in tqdm(enumerate(Exams)):
        temp = table[table.StudyInstanceUID == Exam]
        temp = reduce_resolution_CT(temp, max_resol)
        #distinguish slices
        temp = distinct_slices(temp, window_size)
        l_res = pd.concat([l_res, temp])
    l_res = l_res.reset_index().drop(['index'], axis = 1)
    return(l_res)

In [7]:
tab = pd.read_csv('/media/aous/Samsung_T5/mtrain.csv')
repart = list(tab['StudyInstanceUID'].unique())
test_exams = repart[:1000]
train_exams = repart[1000:]
traindf = tab[tab['StudyInstanceUID'].isin(train_exams)]
testdf = tab[tab['StudyInstanceUID'].isin(test_exams)]
tij = table_3D(testdf, max_resol = 192, window_size = 48)

/home/aous/miniconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/aous/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/aous/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/hom

In [8]:
tij.to_csv('mtest_192_48.csv')

In [7]:
tij = pd.read_csv('mtest_192_48.csv')

In [9]:
def load_detect(net):
    big_detect_net = net
    big_detect_net.load_state_dict(torch.load('best_Bigdetect.pth'))
    big_detect_net = big_detect_net.eval()
    #fscore 50% (good recall)
    return(big_detect_net.cuda())
def load_right(net):
    right_det_net = net
    right_det_net.load_state_dict(torch.load('PEright_45_49.pth'))
    right_det_net = right_det_net.eval()
    #detection: 45% + right: 49%
    return(right_det_net.cuda())
def load_left(net):
    left_net = net
    left_net.load_state_dict(torch.load('best_left.pth'))
    left_net = left_net.eval()
    #fscore 48% acc 91%
    return(left_net.cuda())
def load_central(net):
    central_net = net
    central_net.load_state_dict(torch.load('best_central.pth'))
    central_net = central_net.eval()
    #fscore 57%
    return(central_net.cuda())
models = {'big_detect_net': load_detect, 'right_det_net': load_right,
          'left_net': load_left, 'central_net': load_central}
modelslist = ['big_detect_net', 'right_det_net', 'left_net', 'central_net']
C = {'big_detect_net': ['negative_detect', 'positive_detect'], 'right_det_net' : ['right_detect', 'right'],
    'left_net': ['negative_left', 'positive_left'], 'central_net': ['negative_central', 'positive_central']}

In [10]:
def empty_pipeline(tab):
    eval_tab = pd.DataFrame(columns = ['Exam', 'slice', 'negative_detect', 'positive_detect', 
                                      'right_detect', 'right', 'negative_left', 'positive_left',
                                      'negative_central', 'positive_central', 'negative_chronic','chronic',
                                     'acute', 'qa_motion', 'qa_contrast', 'flow_artifact', 'true_filling_defect_not_pe'])
    Exams = tab['StudyInstanceUID'].dropna().unique()
    for i, exam in tqdm(enumerate(Exams)):
        B1 = [0]*17
        B2 = [0]*17
        B3 = [0]*17
        B4 = [0]*17
        ex = tab[tab.StudyInstanceUID == exam].reset_index().drop(['index'], axis = 1)
        slices = ex['slice'].dropna().unique()
        B1[0] = exam
        B2[0] = exam
        B3[0] = exam
        B4[0] = exam
        B1[1] = slices[0]
        B2[1] = slices[1]
        try:
            B3[1] = slices[2]
        except:
            B3[1] = 'noslice'
        try:
            B4[1] = slices[3]
        except:
            B4[1] = 'noslice'
        B1[13] = ex['qa_motion'][0]
        B1[14] = ex['qa_contrast'][0]
        B1[15] = ex['flow_artifact'][0]
        B1[16] = ex['true_filling_defect_not_pe'][0]
        B2[13] = ex['qa_motion'][0]
        B2[14] = ex['qa_contrast'][0]
        B3[15] = ex['flow_artifact'][0]
        B4[16] = ex['true_filling_defect_not_pe'][0]
        B3[13] = ex['qa_motion'][0]
        B3[14] = ex['qa_contrast'][0]
        B3[15] = ex['flow_artifact'][0]
        B3[16] = ex['true_filling_defect_not_pe'][0]
        B4[13] = ex['qa_motion'][0]
        B4[14] = ex['qa_contrast'][0]
        B4[15] = ex['flow_artifact'][0]
        B4[16] = ex['true_filling_defect_not_pe'][0]
        
        B1 = pd.DataFrame([B1], columns= eval_tab.columns)
        B2 = pd.DataFrame([B2], columns= eval_tab.columns)
        B3 = pd.DataFrame([B3], columns= eval_tab.columns)
        B4 = pd.DataFrame([B4], columns= eval_tab.columns)
        
        eval_tab = pd.concat([eval_tab, B1, B2, B3, B4])
    return(eval_tab.reset_index().drop(['index'], axis = 1))

In [11]:
eval_tab = empty_pipeline(tij)

In [19]:
eval_tab.to_csv('test_eval_tab.csv')

In [11]:
eval_tab = pd.read_csv('test_eval_tab.csv')

In [13]:
for col in eval_tab.columns[3:14]:
    eval_tab[col] = pd.to_numeric(eval_tab[col], downcast="float")

In [14]:
def form_batch(inpt_slices, outpt_size, size, augment):
    inpt_slices.sort_values(['path'], inplace = True)
    transform = transforms.Compose([
                  transforms.ToTensor(),
                  transforms.Normalize(mean=[0.43216, 0.394666, 0.37645],
                                        std=[0.22803, 0.22145, 0.216989])
                ])
    outpt = torch.zeros((3, outpt_size, size[0], size[1]))
    rot = (np.random.random()*2-1)*15
    c = int((np.random.random()*2-1)*size[0]*0.15)
    f = int((np.random.random()*2-1)*size[1]*0.15)
    for i, idx in enumerate(inpt_slices.index):
        im = Image.open(inpt_slices['path'][idx])
        if augment:
            im = im.rotate(rot, translate = (c,f))
        im_vect = transform(im)
        outpt[0][i] = im_vect[0]
        outpt[1][i] = im_vect[1]
        outpt[2][i] = im_vect[2]
    return(outpt)

In [15]:
class PE_dataset_pipeline(torch.utils.data.Dataset):
    def __init__(self, eval_tab, tab, window = 48):
        self.slices = list(eval_tab['slice'].unique())
        self.slices.remove('noslice')
        self.window = window
        self.Table = eval_tab
        self.tab = tab
            
    def __getitem__(self, index):
        mini_batch = self.Table[self.Table.slice == self.slices[index]]
        mb = self.tab[self.tab.slice == self.slices[index]]
        Input = form_batch(mb, self.window, (256, 256), False)
        return([Input, mini_batch.index[0]])
    def __len__(self):
        return(len(self.slices))

In [16]:
def fill_eval_tab(model_name, eval_tab, tij):
    dataset = PE_dataset_pipeline(eval_tab, tij)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size = 4, num_workers = 2)
    #load model
    if model_name == 'chronic_net':
        net = peNet(3)
        net.load_state_dict(torch.load('best_chronic.pth'))
        net = net.eval().cuda()
    else:
        net = peNet(2)
        net = models[model_name](net)
    #fill table
    for i, (inpt, index) in tqdm(enumerate(dataloader)):
        outpt = net(inpt)
        for j, idx in enumerate(index):
            if model_name == 'chronic_net':
                eval_tab['negative_chronic'][idx.item()] = outpt[j][0].item()
                eval_tab['chronic'][idx.item()] = outpt[j][1].item()
                eval_tab['acute'][idx.item()] = outpt[j][2].item()
            else:
                eval_tab[C[model_name][0]][idx.item()] = outpt[j][0].item()
                eval_tab[C[model_name][1]][idx.item()] = outpt[j][1].item()
        outpt = None

In [17]:
C = {'big_detect_net': ['negative_detect', 'positive_detect'], 'right_det_net' : ['right_detect', 'right'],
    'left_net': ['negative_left', 'positive_left'], 'central_net': ['negative_central', 'positive_central']}

In [18]:
fill_eval_tab('chronic_net', eval_tab, tij)
fill_eval_tab('big_detect_net', eval_tab, tij)
fill_eval_tab('right_det_net', eval_tab, tij)
fill_eval_tab('left_net', eval_tab, tij)
fill_eval_tab('central_net', eval_tab, tij)

/home/aous/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/aous/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/aous/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/aous/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18:

/home/aous/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/aous/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/aous/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/aous/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22:

/home/aous/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
eval_tab.to_csv('eval_tab_filled.csv')

In [ ]:
class PE_dataset_pipeline(torch.utils.data.Dataset):
    def __init__(self, Table, eval_tab, window = 48):
        self.slices = list(eval_tab['slice'].unique())
        self.window = window
        self.Table = Table
            
    def __getitem__(self, index):
        mini_batch = self.Table[self.Table.slice == self.slices[index]].reset_index().drop(['index'], axis = 1)
        Study = mini_batch['Exam'][0]
        temp = self.Table[self.Table.StudyInstanceUID == Study]
        Input = form_batch(mini_batch, self.window, (256, 256), self.augmented)
        
        Target = torch.zeros(9)
        pe_num = len(mini_batch[mini_batch.pe_present_on_image == 1])
        Target[0] = mini_batch['indeterminate'][0]
        Target[1] = mini_batch['negative_exam_for_pe'][0]
        Target[2] = mini_batch['central_pe'][0]
        Target[3] = mini_batch['rightsided_pe'][0]
        Target[4] = mini_batch['leftsided_pe'][0]
        Target[5] = mini_batch['chronic_pe'][0]
        Target[6] = mini_batch['acute_and_chronic_pe'][0]
        Target[7] = mini_batch['rv_lv_ratio_gte_1'][0]
        Target[8] = mini_batch['rv_lv_ratio_lt_1'][0]
        return([Input, Target])
    def __len__(self):
        return(len(self.slices))

In [9]:
def pipeline(tab):
    eval_tab = pd.DataFrame(columns = ['Exam', 'negative_detect', 'positive_detect', 
                                      'right_detect', 'right', 'negative_left', 'positive_left',
                                      'negative_central', 'positive_central', 'negative_chronic','chronic',
                                     'acute', 'qa_motion', 'qa_contrast', 'flow_artifact', 'true_filling_defect_not_pe'])
    Exams = tab['StudyInstanceUID'].dropna().unique()
    #non chronic: 86% chronic: 15% acute: 26%
    for i, exam in tqdm(enumerate(Exams)):
        B1 = [0]*16
        B2 = [0]*16
        B3 = [0]*16
        B4 = [0]*16
        ex = tab[tab.StudyInstanceUID == exam].reset_index().drop(['index'], axis = 1)
        slices = ex['slice'].dropna().unique()
        B1[0] = exam
        B2[0] = exam
        B3[0] = exam
        B4[0] = exam
        inpt = torch.zeros((4, 3, 48, 256, 256)).cuda()
        for i, sl in enumerate(slices[:4]):
            batch = form_batch(tab[tab.slice == sl], 48, (256,256), False)
            inpt[i] = batch
        for i, model in enumerate(modelslist):
            net = peNet(2)
            net = models[model](net)
            outpt = net(inpt)
            B1[2*i+1] = outpt[0][0].item()
            B1[2*i+2] = outpt[0][1].item()
            B2[2*i+1] = outpt[1][0].item()
            B2[2*i+2] = outpt[1][1].item()
            B3[2*i+1] = outpt[2][0].item()
            B3[2*i+2] = outpt[2][1].item()
            B4[2*i+1] = outpt[3][0].item()
            B4[2*i+2] = outpt[3][1].item()
            outpt = None
            net = None
            batch = None
        net = peNet(3)
        net.load_state_dict(torch.load('best_chronic.pth'))
        net = net.eval().cuda()
        outpt = net(inpt)
        B1[9] = outpt[0][0].item()
        B1[10] = outpt[0][1].item()
        B1[11] = outpt[0][2].item()
        B1[12] = ex['qa_motion'][0]
        B1[13] = ex['qa_contrast'][0]
        B1[14] = ex['flow_artifact'][0]
        B1[15] = ex['true_filling_defect_not_pe'][0]
        B2[9] = outpt[1][0].item()
        B2[10] = outpt[1][1].item()
        B2[11] = outpt[1][2].item()
        B2[12] = ex['qa_motion'][0]
        B2[13] = ex['qa_contrast'][0]
        B3[14] = ex['flow_artifact'][0]
        B4[15] = ex['true_filling_defect_not_pe'][0]
        B3[9] = outpt[2][0].item()
        B3[10] = outpt[2][1].item()
        B3[11] = outpt[2][2].item()
        B3[12] = ex['qa_motion'][0]
        B3[13] = ex['qa_contrast'][0]
        B3[14] = ex['flow_artifact'][0]
        B3[15] = ex['true_filling_defect_not_pe'][0]
        B4[9] = outpt[3][0].item()
        B4[10] = outpt[3][1].item()
        B4[11] = outpt[3][2].item()
        B4[12] = ex['qa_motion'][0]
        B4[13] = ex['qa_contrast'][0]
        B4[14] = ex['flow_artifact'][0]
        B4[15] = ex['true_filling_defect_not_pe'][0]
        outpt = None
        net = None
        batch = None
        
        B1 = pd.DataFrame([B1], columns= eval_tab.columns)
        B2 = pd.DataFrame([B2], columns= eval_tab.columns)
        B3 = pd.DataFrame([B3], columns= eval_tab.columns)
        B4 = pd.DataFrame([B4], columns= eval_tab.columns)
        
        eval_tab = pd.concat([eval_tab, B1, B2, B3, B4])
    return(eval_tab.reset_index().drop(['index'], axis = 1))